In [1]:
import os, sys, json
cwd = os.getcwd()
PROJECT_PATH = cwd[:cwd.find('midfreq')+7]
print('PROJECT_PATH:', PROJECT_PATH)
if PROJECT_PATH not in sys.path: sys.path.append(PROJECT_PATH)
os.chdir(PROJECT_PATH) 

from datetime import datetime, timedelta, timezone
import pandas as pd

PROJECT_PATH: /home/alex/PycharmProjects/midfreq


In [2]:
import plotly.express as px
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# read files

In [3]:
# vault = 'DynYieldB'
vault = 'DynLst'
# dirname = f"C:/Users/david/singularity/midfreq/scrappers/defillama_history/data/{vault}"
dirname = f"{PROJECT_PATH}/data/{vault}"
print(dirname)
pools = []
for filename in os.listdir(dirname):
    fullname = os.path.join(os.sep, dirname, filename)
    print(fullname)
    data = pd.read_csv(fullname, parse_dates=['date']).set_index('date')['haircut_apy']
    pools.append(data.rename(filename.replace('.csv','')))
history = pd.concat(pools, axis=1)

/home/alex/PycharmProjects/midfreq/data/DynLst
/home/alex/PycharmProjects/midfreq/data/DynLst/6d342d6a-9796-443e-8152-b94e8b6021fc.csv
/home/alex/PycharmProjects/midfreq/data/DynLst/77020688-e1f9-443c-9388-e51ace15cc32.csv
/home/alex/PycharmProjects/midfreq/data/DynLst/ce70e358-6668-498c-9afa-651f770eac27.csv
/home/alex/PycharmProjects/midfreq/data/DynLst/42c02674-f7a2-4cb0-be3d-ade268838770.csv
/home/alex/PycharmProjects/midfreq/data/DynLst/747c1d2a-c668-4682-b9f9-296708a3dd90.csv
/home/alex/PycharmProjects/midfreq/data/DynLst/5ce23e7e-3800-4c9c-ad30-6db3db0515a1.csv
/home/alex/PycharmProjects/midfreq/data/DynLst/e378a7c2-6285-4993-9397-87ac9c8adc15.csv
/home/alex/PycharmProjects/midfreq/data/DynLst/a4b5b995-99e7-4b8f-916d-8940b5627d70.csv
/home/alex/PycharmProjects/midfreq/data/DynLst/4cc5df76-f81d-49fe-9e1e-4caa6a8dad0b.csv
/home/alex/PycharmProjects/midfreq/data/DynLst/c9873dab-0979-478c-b48c-3c3a0c935449.csv
/home/alex/PycharmProjects/midfreq/data/DynLst/98463e86-e32c-4998-830c-cc

In [4]:
# metadata = pd.read_csv(f"C:/Users/david/singularity/midfreq/scrappers/defillama_history/data/{vault}_pool_metadata.csv")
metadata = pd.read_csv(f"{PROJECT_PATH}/data/{vault}_pool_metadata.csv")
pool_map = metadata[['pool','project','symbol']].set_index('pool').T.to_dict()
pool_map = {key: '{}_{}'.format(x['project'], x['symbol']) for key, x in pool_map.items()}
index_map = {f'weight_{i}': f'w_{value}' for i, value in enumerate(pool_map.values())}
index_map |= {f'yield_{i}': f'y_{value}' for i, value in enumerate(pool_map.values())} 
pool_map

{'747c1d2a-c668-4682-b9f9-296708a3dd90': 'lido_STETH',
 'd4b3c522-6127-4b89-bedf-83641cdcd2eb': 'rocket-pool_RETH',
 'e6435aae-cbe9-4d26-ab2c-a4d533db9972': 'aave-v3_WSTETH',
 '77020688-e1f9-443c-9388-e51ace15cc32': 'frax-ether_SFRXETH',
 'e880e828-ca59-4ec6-8d4f-27182a4dc23d': 'aave-v3_WETH',
 '5ce23e7e-3800-4c9c-ad30-6db3db0515a1': 'convex-finance_ETH-STETH',
 '6d342d6a-9796-443e-8152-b94e8b6021fc': 'convex-finance_ETH-FRXETH',
 'e378a7c2-6285-4993-9397-87ac9c8adc15': 'convex-finance_ETH-STETH',
 'f1a22245-a1f1-4389-8ff2-dde4914b272d': 'aave-v3_RETH',
 '42c02674-f7a2-4cb0-be3d-ade268838770': 'balancer-v2_RETH-WETH',
 'a4b5b995-99e7-4b8f-916d-8940b5627d70': 'aura_RETH-WETH',
 'ce70e358-6668-498c-9afa-651f770eac27': 'aura_WSTETH-SFRXETH-RETH',
 '4fc7d3dc-a6f0-40e6-af53-ab9d6b50b5c0': 'convex-finance_WETH-FRXETH',
 '98463e86-e32c-4998-830c-ccd47dbb3041': 'aura_WSTETH-WETH',
 'c9873dab-0979-478c-b48c-3c3a0c935449': 'convex-finance_STETH-FRXETH',
 '4cc5df76-f81d-49fe-9e1e-4caa6a8dad0b': '

# compute historical pool rank

In [5]:
max_rank = 10
end = datetime.now().replace(tzinfo=timezone.utc)
start = end - timedelta(days = 90)
df = history.fillna(method='ffill')
df = df[(df.index >= start)&(df.index <= end)]
best_only = df[df.rank(axis=1, ascending=False)<max_rank].dropna(how='all', axis=1)
#best_only = best_only.rename(columns=pool_map)
best_only.iplot()

In [6]:
# with open(os.path.join(os.sep, dirname, f'EverTop{max_rank}.json'), 'w') as fp:
#     json.dump(list(best_only.columns), fp=fp)

In [7]:
short_listed_protocols = [
                'lido',
            'frax-ether',
            'rocket-pool',
            'aave-v3',
            'morpho-aavev3',
            'balancer-v2',
            'curve-finance',
            'convex-finance',
            'aura',
            'compound',
            'morpho-compound'
            'compound-v3',
            'morpho-aave',
            'convex-finance',
            'uniswap-v3',
            'uniswap-v2',
            'aave-v2',
            'aave-v3',
            'curve-dex',
            'spark',
            'makerdao',
            'aura',
            'balancer',
'stargate',
                                                 'venus',
                                                 'pancakeswap-amm-v3',
                                                 'wing-finance',
                                                 'thena-v1',
                                                 'pancakeswap-amm']
for it in best_only.columns:
    descr = pool_map[it] if it in pool_map else []
    if any(protocol in descr for protocol in short_listed_protocols):
        print(f'- \"{it}\" # {descr}')

- "6d342d6a-9796-443e-8152-b94e8b6021fc" # convex-finance_ETH-FRXETH
- "77020688-e1f9-443c-9388-e51ace15cc32" # frax-ether_SFRXETH
- "ce70e358-6668-498c-9afa-651f770eac27" # aura_WSTETH-SFRXETH-RETH
- "42c02674-f7a2-4cb0-be3d-ade268838770" # balancer-v2_RETH-WETH
- "747c1d2a-c668-4682-b9f9-296708a3dd90" # lido_STETH
- "e378a7c2-6285-4993-9397-87ac9c8adc15" # convex-finance_ETH-STETH
- "a4b5b995-99e7-4b8f-916d-8940b5627d70" # aura_RETH-WETH
- "4cc5df76-f81d-49fe-9e1e-4caa6a8dad0b" # balancer-v2_WSTETH-WETH
- "c9873dab-0979-478c-b48c-3c3a0c935449" # convex-finance_STETH-FRXETH
- "98463e86-e32c-4998-830c-ccd47dbb3041" # aura_WSTETH-WETH
- "d4b3c522-6127-4b89-bedf-83641cdcd2eb" # rocket-pool_RETH
- "4fc7d3dc-a6f0-40e6-af53-ab9d6b50b5c0" # convex-finance_WETH-FRXETH
- "fc7701fc-e290-4606-a8be-9e4ba6c5f91a" # convex-finance_ETH-RETH
- "f1a22245-a1f1-4389-8ff2-dde4914b272d" # aave-v3_RETH
- "e6435aae-cbe9-4d26-ab2c-a4d533db9972" # aave-v3_WSTETH
